# Constraint satisfaction problem

In the previous tasks the state was opaque: an algorithm could execute an action and transform a state to another state, but that was it. It could not depend in its search on the details of a state, because the state representation was different from problem to problem. In CSP this is no longer true: the state always consists of an assignment of values over some domains to variables. This enables an algorithm to interact with the state directly, not only through opaque actions. 

Allowed assignments are limited by constraints in the problem and broadly speaking they can be divided into three groups:

* *Unary constraints* each concerning only a single variable (e.g., $x>0$)
* *Binary constraints* relating two variables to each other (e.g., $x>y$)
* *General constraints* concerning more than two variables at once

In general, the shape of constraints is unrestricted and we are not limited to numeric domains and constraints in the form of inequalities. In principle, one could employ arbitrary boolean functions as constraints.

The following class `CSP` is a very limited representation of such a problem capable only of handling:

* finite domains that can be materialized (e.g., the set of all 32-bit integers is doable in theory; the set of all 64-bit integers is out of reach) 
* unary constraints by setting the domain of a variable appropriately;
* binary constraints defined in terms of Python's operators `==`, `!=`, `<`, `<=`, `>` and `>=` such that both sides are variables.

About its methods:

* The constructor (`__init__`) requires no arguments.
* `addVariable` requires two arguments: 
    * `var` - a variable (a string representing its name, but really, it could be any object);
    * `domain` - a collection of the allowed values for the variable
* `addBinaryConstraint` also requires three arguments:
    * `var1` - a variable, the left-hand side of the constraint
    * `op` - an operator (one of the following strings: `'=='`, `'!='`, `'<'`, `'<='`, `'>'`, `'>='`)
    * `var2` - a variable, the right-hand side of the constraint
* `verify` is used to verify whether the relation given by the second argument `op` holds between the values given as the first argument `left` and the third argument `right`. Returns `True` if it does and `False` otherwise.
* `is_complete` gets a single argument: a dictionary representing an assignment of values to variables. From now on, we call such a dictionary an *assignment*. It returns `True` if all the variables of the problem are present in the dictionary, and `False` otherwise.
* `is_consistent` also requries an assignment. It returns `True` if all the values in the assignment are such that they come from the respecitve domains, and no binary constrain is violated (but if the assignment is not complete then some may be unfulfilled).

In [36]:
class CSP:
    def __init__(self):
        self.domains = {}
        self.binary = []
    
    def addVariable(self, var, domain):        
        assert var not in self.domains
        self.domains[var] = set(domain)
        
    def addBinaryConstraint(self, var1, operator, var2):
        assert var1 in self.domains
        assert var2 in self.domains
        c = (var1, operator, var2)
        self.binary.append(c)      
        
    def verify(self, left, op, right):
        if op[0] == '=':
            return left == right
        if op == '!=':
            return left != right
        if op == '<':
            return left < right
        if op == '<=':
            return left <= right
        if op == '>':
            return left > right
        if op == '>=':
            return left >= right
        
    def is_complete(self, assignment):
        return self.domains.keys() <= assignment.keys() 
        
    def is_consistent(self, assignment):
        for var, value in assignment.items():            
            if value not in self.domains[var]:
                return False
        for var1, op, var2 in self.binary:
            if var1 in assignment and var2 in assignment:
                if not self.verify(assignment[var1], op, assignment[var2]):
                    return False
        return True

## Making Australia colorful again

Lets model a map coloring problem. In particular, we'll color the following map of Australia. It can be simplified to a graph presented in the right-hand side of the picture below.

![](aima-images/fig_6_1.png)

We begin by creating a new problem in the variable `australia` over the domain consisting of three values: `'R'` statnding for the color red, `'G'` standing for green and `'B'` standing for blue. Of course the particular symbols used are not relevant. There are 7 teritories, so we add 7 variables to the problem.

In [37]:
australia = CSP()
australia.addVariable('WA', {'R', 'G', 'B'})
australia.addVariable('NT', {'R', 'G', 'B'})
australia.addVariable('SA', {'R', 'G', 'B'})
australia.addVariable('Q', {'R', 'G', 'B'})
australia.addVariable('NSW', {'R', 'G', 'B'})
australia.addVariable('V', {'R', 'G', 'B'})
australia.addVariable('T', {'R', 'G', 'B'})

Any two teritories of Australia sharing a border should be colored using different colours. We thus define 9 binary constraints.

In [38]:
australia.addBinaryConstraint('WA', '!=', 'NT')
australia.addBinaryConstraint('WA', '!=', 'SA')
australia.addBinaryConstraint('NT', '!=', 'SA')
australia.addBinaryConstraint('NT', '!=', 'Q')
australia.addBinaryConstraint('SA', '!=', 'Q')
australia.addBinaryConstraint('SA', '!=', 'NSW')
australia.addBinaryConstraint('SA', '!=', 'V')
australia.addBinaryConstraint('Q', '!=', 'NSW')
australia.addBinaryConstraint('NSW', '!=', 'V')

Let's see how the `verify` method works.

In [39]:
print("Is G=G?", australia.verify('G', '==', 'G'))
print("Is 5<7?", australia.verify(5, '<', 7))
print("Is 5>=7?", australia.verify(5, '>=', 7))

Is G=G? True
Is 5<7? True
Is 5>=7? False


Lets now consider an assignment such that *Western Australia* is red and the rest is unspecified. Is it a consistent assignment?

In [40]:
australia.is_consistent({'WA': 'R'})

True

Is it a complete assignment?

In [41]:
australia.is_complete({'WA': 'R'})

False

As expected it didn't work: there are still 9 unfulfiled constraints. So lets extend the solution.

In [42]:
assignment = {'WA': 'R', 'Q': 'R', 'V': 'R', 'NT': 'B', 'NSW': 'B', 'SA': 'G'}
print("Is consistent?", australia.is_consistent(assignment))
print("Is complete?", australia.is_complete(assignment))

Is consistent? True
Is complete? False


Now it almost works: all the constraints are satisfied, but the solution is still not complete, as Tasmania is still colorless.

In [43]:
assignment = {'WA': 'R', 'Q': 'R', 'V': 'R', 'NT': 'B', 'NSW': 'B', 'SA': 'G', 'T': 'B'}
print("Is consistent?", australia.is_consistent(assignment))
print("Is complete?", australia.is_complete(assignment))

Is consistent? True
Is complete? True


Finally, we have manually collored the map of Australia! Now let's make computer do this for us.

## Task 1: Backtracking

Complete the following cell of code by implementing the recursive backtracking algorithm. Basically, it is a recursive DFS where an action is assigning a value to a variable. Avoid making copies of assignments, instead operate on a single assignment, and update it accordingly. Increment `self.counter` every time you check wheter an assigment is a solution (i.e., is consistent and complete). Use the provided methods `select_unassigned_variable` and `order_domain_values` to, respectively, get a next variable to assign a value to, and to get the list of allowed values for the variable. `solve` should return a solution (i.e., a complete, consistent assignment) or `None`.

In [44]:
class RecursiveBacktracking:
    def __init__(self, csp):
        self.csp = csp
        self.counter = 0
        
    def select_unassigned_variable(self, assignment):
        for var in self.csp.domains.keys():
            if var not in assignment:
                return var
        return None
    
    def order_domain_values(self, variable, assignment):
        return self.csp.domains[variable]

    def backtracking(self, assignment):
        # self.counter += 1
        if self.csp.is_complete(assignment):
          return assignment

        new_variable = self.select_unassigned_variable(assignment)
        for i in self.order_domain_values(new_variable, assignment):
            assignment[new_variable] = i
            self.counter += 1
            if self.csp.is_consistent(assignment):
                new_assignment = self.backtracking(assignment)
                if new_assignment is not None:
                    return new_assignment
            del assignment[new_variable]
        return None
        
    
    def solve(self):
        self.counter = 0
        return self.backtracking({})

Let's test it on the Australia problem! In the cell below you should get a consistent, complete assignment, i.e., a solution to the problem.

In [45]:
solver = RecursiveBacktracking(australia)
assignment = solver.solve()
print("Assignment", assignment)
print("Is consistent?", australia.is_consistent(assignment))
print("Is complete?", australia.is_complete(assignment))
print("# considered assignments", solver.counter)

Assignment {'WA': 'B', 'NT': 'G', 'SA': 'R', 'Q': 'B', 'NSW': 'G', 'V': 'B', 'T': 'B'}
Is consistent? True
Is complete? True
# considered assignments 11


## Task 2: Extend `RecursiveBacktracking` with the MRV and degree heuristics

Modify your `RecursiveBacktracking` to include the MRV (minimum remaining values) heuristics and the degree heuristics. Implement them in the appropriate places below in the class `RecursiveBacktrackingWithHeuristics`. If you want, you can also implement the least constraining value heuristics, but this is not mandatory. Follow the same assumptions as for `RecursiveBacktracking`, so expose the main backtracking algorithm as the `solve` and keep track of the number of considered assignments in the `counter` variable.

In [46]:
from copy import deepcopy
from queue import PriorityQueue

class RecursiveBacktrackingWithHeuristics:
    def __init__(self, csp):
        self.csp = csp
        self.counter = 0
        
    def select_unassigned_variable(self, assignment, domains):
        # MRV
        mini = min(domains, key=lambda x: len(domains[x]))
        minis = [mini]
        for i in domains:
            if i not in assignment and i != mini and len(domains[i]) == len(domains[mini]):
                minis.append(i)
        if len(minis) == 1:
            return mini
        # degree heuristic
        counts = {}
        for key in minis:
            counts[key] = 0
        for i in self.csp.binary:
            if i[0] not in assignment and i[2] not in assignment and i[0]:
                if i[0] in minis:
                    counts[i[0]] += 1
                if i[2] in minis:
                    counts[i[2]] += 1
        return max(counts, key=counts.get)
    
    def order_domain_values(self, variable, domains, assignment):
        order = PriorityQueue()
        for d in domains[variable]:
            counters = []
            for i in self.csp.binary:
                if i[0] not in assignment and i[2] not in assignment:
                    cnt = 0
                    if i[0] == variable:
                        for j in domains[i[2]]:
                            if self.csp.verify(d, i[1], j):
                                cnt += 1
                        counters.append(cnt)
                    elif i[2] == variable:
                        for j in domains[i[0]]:
                            if self.csp.verify(j, i[1], d):
                                cnt += 1
                        counters.append(cnt)
            # if out variable has no constraints, then no matter what value we will assign it will be good, so return first value from domain
            if len(counters) == 0:
                return [d]
            order.put((min(counters), d))
        return reversed(list(x[1] for x in order.queue))
 
    def delete_value_from_degree(self, variable, value, domains):
        del domains[variable]
        for i in self.csp.binary:
            if i[0] == variable and i[2] in domains.keys():
                pom = list(domains[i[2]])
                for d in pom:
                    if self.csp.verify(value, i[1], d) == False:
                        domains[i[2]].remove(d)

            if i[2] == variable and i[0] in domains.keys():
                pom = list(domains[i[0]])
                for d in pom:
                    if self.csp.verify(d, i[1], value) == False:
                        domains[i[0]].remove(d)
        return domains

    def backtracking(self, assignment, domains):
        # self.counter += 1
        if self.csp.is_complete(assignment):
          return assignment

        new_variable = self.select_unassigned_variable(assignment, domains)
        for i in self.order_domain_values(new_variable, domains, assignment):
            assignment[new_variable] = i
            self.counter += 1
            if self.csp.is_consistent(assignment):
                new_assignment = self.backtracking(assignment, self.delete_value_from_degree(new_variable, i, deepcopy(domains)))
                if new_assignment is not None:
                    return new_assignment
            del assignment[new_variable]
        return None
        
    def solve(self):
        self.counter = 0
        domains = deepcopy(self.csp.domains)
        assignment = {}
        # before start I look if there are some variables with only one possible value to assign from domain, if there is such one I add it to assignment
        for i in self.csp.domains:
            if len(domains[i]) == 1:
                assignment[i] = list(domains[i])[0]
                domains = self.delete_value_from_degree(i, list(domains[i])[0], domains)
        return self.backtracking(assignment, domains)

Lets test your implementation by solving the problem of coloring Australia!

In [47]:
solver = RecursiveBacktrackingWithHeuristics(australia)
assignment = solver.solve()
print("Assignment", assignment)
print("Is consistent?", australia.is_consistent(assignment))
print("Is complete?", australia.is_complete(assignment))
print("# considered assignments", solver.counter)

Assignment {'SA': 'G', 'NT': 'R', 'Q': 'B', 'NSW': 'R', 'WA': 'B', 'V': 'B', 'T': 'R'}
Is consistent? True
Is complete? True
# considered assignments 7


**Compare the number of considered assignments for `RecursiveBacktracking` and `RecursiveBacktrackingWithHeuristics`. How do they differ? Why do you think is that?**

Number of considered assignments differs by 4 (11 in RecursiveBacktracking and 7 in RecursiveBacktrackingWithHeuristic). With heuristics there is less, because by choosing proper variable to assign value there are weak chances that we will have to backtrack and of cource also choosing proper value from domain to assign our chances to backtrack are smaller.

## Sudoku!

Coloring Australia is really a bit boring. Lets deal with something more exciting: SUDOKU! I assume you know the rules, but if you don't, go look them up, spend some time solving a few puzzles and familiarize yourself with the puzzle.
The cell below defines a string `puzzle` which represents a Sudoku puzzle from the picture below. A digit in `puzzle` corresponds to a digit in the puzzle, while an underscore `_` corresponds to a blank cell in the puzzle. For your convenience there's also variable `solution` containing the correct solution to this particular puzzle.

![](aima-images/fig_6_4.png)

In [48]:
# level 1
puzzle = '''
__3_2_6__
9__3_5__1
__18_64__
__81_29__
7_______8
__67_82__
__26_95__
8__2_3__9
__5_1_3__
'''

# level 2
# puzzle = '''
# 9_____7__
# _4__3____
# __8__2_91
# 8__9_____
# _______6_
# __9__1_25
# 3___2__57
# __5___6__
# _____78__
# '''

# level 3
# puzzle = '''
# 8________
# __36_____
# _7__9_2__
# _5___7___
# ____457__
# ___1___3_
# __1____68
# __85___1_
# _9____4__
# '''

In [49]:
solution = '''
483921657
967345821
251876493
548132976
729564138
136798245
372689514
814253769
695417382
'''

## Task 3: Sudoku as CSP

Using the string `puzzle` defined above write in the cell below code that will add all the necessary constraints to the CSP represented by the variable `sudoku`. If you can and want, write a parser that will construct the CSP from a string representation of the puzzle. Otherwise, you can construct the problem manually.

In [50]:
sudoku = CSP()
p=0
for i in range(len(puzzle)):
    if puzzle[i] != '\n':
        if puzzle[i] == '_':
            sudoku.addVariable((p//9, p%9), {'1', '2', '3', '4', '5', '6', '7', '8', '9'})
        else:
            sudoku.addVariable((p//9, p%9), puzzle[i])
        p += 1

for i in range(9):
    for j in range(9):
        # add constraint in square
        if (i%3,j%3) in [(0,0), (0,2), (2,0), (2,2)]:
            p=1
            if i%3 == 2:
                p = -1
            k=1
            if j%3 == 2:
                k = -1
            sudoku.addBinaryConstraint((i,j), '!=', (i+(1*p), j+(1*k)))
            sudoku.addBinaryConstraint((i,j), '!=', (i+(1*p), j+(2*k)))
            sudoku.addBinaryConstraint((i,j), '!=', (i+(2*p), j+(1*k)))
            sudoku.addBinaryConstraint((i,j), '!=', (i+(2*p), j+(2*k)))
        elif (i%3,j%3) in [(0,1), (1,0), (1,2), (2,1)]:
            p1=1
            p2=2
            if i%3 == 1:
                p1 = -1
                p2 = 1
            elif i%3 == 2:
                p1 = -1
                p2 = -2

            k1=1
            k2=2
            if j%3 == 1:
                k1 = -1
                k2 = 1
            elif j%3 == 2:
                k1 = -2
                k2 = -1

            sudoku.addBinaryConstraint((i,j), '!=', (i+p1, j+k1))
            sudoku.addBinaryConstraint((i,j), '!=', (i+p2, j+k1))
            sudoku.addBinaryConstraint((i,j), '!=', (i+p1, j+k2))
            sudoku.addBinaryConstraint((i,j), '!=', (i+p2, j+k2))
        else:
            sudoku.addBinaryConstraint((i,j), '!=', (i-1, j-1))
            sudoku.addBinaryConstraint((i,j), '!=', (i-1, j+1))
            sudoku.addBinaryConstraint((i,j), '!=', (i+1, j-1))
            sudoku.addBinaryConstraint((i,j), '!=', (i+1, j+1))
        for z in range(j, 9):
            if (i, j) != (i, z):
                # add contraint in rows
                sudoku.addBinaryConstraint((i,j), '!=', (i, z))
                # add constraint in columns
                sudoku.addBinaryConstraint((j, i), '!=', (z, i))

Let's see if it works... Unless you did some clever tricks and/or are quite patient - it does not, it is too slow.

In [51]:
def print_sudoku(assignment):
    for i in range(9):
        for j in range(9):
            print(assignment[i,j], end=" ")
        print()

In [52]:
solver = RecursiveBacktrackingWithHeuristics(sudoku)
assignment = solver.solve()
print("Assignment", print_sudoku(assignment))
print("Is consistent?", sudoku.is_consistent(assignment))
print("Is complete?", sudoku.is_complete(assignment))
print("# considered assignments", solver.counter) 

4 8 3 9 2 1 6 5 7 
9 6 7 3 4 5 8 2 1 
2 5 1 8 7 6 4 9 3 
5 4 8 1 3 2 9 7 6 
7 2 9 5 6 4 1 3 8 
1 3 6 7 9 8 2 4 5 
3 7 2 6 8 9 5 1 4 
8 1 4 2 5 3 7 6 9 
6 9 5 4 1 7 3 8 2 
Assignment None
Is consistent? True
Is complete? True
# considered assignments 47


## Task 4: Solving sudoku

In order to make it working, we need to extend backtracking with inference. Complete the following class `RecursiveBacktrackingWithAC3` in such a way that it uses the same heuristics as `RecursiveBacktrackingWithHeuristics` and on top of that uses the AC3 algorithm after a new value is added to the assignment.

In this task, you don't need to stick to the previous API with method. A constructor with a single argument, and an argumentless `solve` are sufficient, heuristics and AC3 can be handled however you want.

In [53]:
class RecursiveBacktrackingWithAC3:
    def __init__(self, csp):
        self.csp = csp
        self.counter = 0

    def reverseOperator(self, op):
        if op[0] == '=':
            return '='
        if op == '!=':
            return '!='
        if op == '<':
            return '>'
        if op == '<=':
            return '>='
        if op == '>':
            return '<'
        if op == '>=':
            return '<='
        
    def removeInconsistentValues(self, i, z, j, domains):
        removed = False
        domainscpy = list(domains[i])
        for x in domainscpy:
            toRemove = True
            for y in domains[j]:
                if self.csp.verify(x, z, y):
                    toRemove = False
                    break
            if toRemove:
                domains[i].remove(x)
                removed = True
        return removed

    def ac3(self, assignment, domains):
        queue = []
        for var1, op , var2 in self.csp.binary:
            if var1 not in assignment and var2 not in assignment:
                queue.append((var1, op, var2))
                queue.append((var2, self.reverseOperator(op), var1))

        while len(queue) > 0:
            (i, z, j) = queue.pop(0)
            if self.removeInconsistentValues(i, z, j, domains):
                neighbors = []
                for var1, op, var2 in self.csp.binary:
                    if var1 not in assignment and var2 not in assignment:
                        if var1 == i:
                            neighbors.append((var2, op))
                        elif var2 == i:
                            neighbors.append((var1, op))
                for k, p in neighbors:
                    queue.append((k, p, i))


    def select_unassigned_variable(self, assignment, domains):
        # MRV
        mini = min(domains, key=lambda x: len(domains[x]))
        minis = [mini]
        for i in domains:
            if i not in assignment and i != mini and len(domains[i]) == len(domains[mini]):
                minis.append(i)
        if len(minis) == 1:
            return mini
        # degree heuristic
        counts = {}
        for key in minis:
            counts[key] = 0
        for i in self.csp.binary:
            if i[0] not in assignment and i[2] not in assignment and i[0]:
                if i[0] in minis:
                    counts[i[0]] += 1
                if i[2] in minis:
                    counts[i[2]] += 1
        return max(counts, key=counts.get)
    
    def order_domain_values(self, variable, domains, assignment):
        order = PriorityQueue()
        for d in domains[variable]:
            counters = []
            for i in self.csp.binary:
                if i[0] not in assignment and i[2] not in assignment:
                    cnt = 0
                    if i[0] == variable:
                        for j in domains[i[2]]:
                            if self.csp.verify(d, i[1], j):
                                cnt += 1
                        counters.append(cnt)
                    elif i[2] == variable:
                        for j in domains[i[0]]:
                            if self.csp.verify(j, i[1], d):
                                cnt += 1
                        counters.append(cnt)
            if len(counters) == 0:
                return [d]
            order.put((min(counters), d))
        return reversed(list(x[1] for x in order.queue))

    def delete_value_from_degree(self, variable, value, domains):
        del domains[variable]
        for i in self.csp.binary:
            if i[0] == variable and i[2] in domains.keys():
                pom = list(domains[i[2]])
                for d in pom:
                    if self.csp.verify(value, i[1], d) == False:
                        domains[i[2]].remove(d)

            if i[2] == variable and i[0] in domains.keys():
                pom = list(domains[i[0]])
                for d in pom:
                    if self.csp.verify(d, i[1], value) == False:
                        domains[i[0]].remove(d)
        return domains

    def backtracking(self, assignment, domains):
        # self.counter += 1
        if self.csp.is_complete(assignment):
          return assignment

        new_variable = self.select_unassigned_variable(assignment, domains)
        for i in self.order_domain_values(new_variable, domains, assignment):
            assignment[new_variable] = i
            self.counter += 1
            if self.csp.is_consistent(assignment):
                domainscpy = deepcopy(domains)
                self.ac3(assignment, domainscpy)
                new_assignment = self.backtracking(assignment, self.delete_value_from_degree(new_variable, i, domainscpy))
                if new_assignment is not None:
                    return new_assignment
            del assignment[new_variable]
        return None
        
    def solve(self):
        self.counter = 0
        domains = deepcopy(self.csp.domains)
        assignment = {}
        for i in self.csp.domains:
            if len(domains[i]) == 1:
                assignment[i] = list(domains[i])[0]
                domains = self.delete_value_from_degree(i, list(domains[i])[0], domains)
        return self.backtracking(assignment, domains)

Let's test it on both considered problem. If everything went all right, both cells should terminate within few seconds at most, and you should see that the number of considered assignments is very small.

In [54]:
solver = RecursiveBacktrackingWithAC3(australia)
assignment = solver.solve()
print("Assignment", assignment)
print("Is consistent?", australia.is_consistent(assignment))
print("Is complete?", australia.is_complete(assignment))
print("# considered assignments", solver.counter)

Assignment {'SA': 'G', 'NT': 'R', 'Q': 'B', 'NSW': 'R', 'WA': 'B', 'V': 'B', 'T': 'R'}
Is consistent? True
Is complete? True
# considered assignments 7


In [55]:
solver = RecursiveBacktrackingWithAC3(sudoku)
assignment = solver.solve()
print("Assignment", print_sudoku(assignment))
print("Is consistent?", sudoku.is_consistent(assignment))
print("Is complete?", sudoku.is_complete(assignment))
print("# considered assignments", solver.counter)
# for the first sudoku algorithm without ac-3 and with ac-3 not differ, but with harder sudoku ac-3 helps to achieve less considered assignmants, but in longer time.
# We talked about it with Michał Kałmucki also in class and in mail.

4 8 3 9 2 1 6 5 7 
9 6 7 3 4 5 8 2 1 
2 5 1 8 7 6 4 9 3 
5 4 8 1 3 2 9 7 6 
7 2 9 5 6 4 1 3 8 
1 3 6 7 9 8 2 4 5 
3 7 2 6 8 9 5 1 4 
8 1 4 2 5 3 7 6 9 
6 9 5 4 1 7 3 8 2 
Assignment None
Is consistent? True
Is complete? True
# considered assignments 47


------------
The pictures are from "Artificial Intelligence: A Modern Approach" 3rd ed.